In [1]:
!pip install lm-eval==0.3.0 -qqq
#!git clone https://github.com/EleutherAI/lm-evaluation-harness

In [2]:
import ctranslate2
import glob
import os
import pandas as pd
import peft
import random
import timeit
import urllib
import torch
import json
import os
from lm_eval import tasks, evaluator, utils
import lm_eval
import wandb
from peft import get_peft_model, LoraConfig, TaskType
from peft import PeftModel, PeftConfig
import wandb.apis.reports as wr
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline

In [3]:
os.environ["WANDB_PROJECT"] = "Autocompletion with evaluation"
os.environ["WANDB_ENTITY"] = "reviewco"
os.environ["WANDB_USERNAME"] = "keisuke-kamata"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
os.environ["WANDB_WATCH"] = "gradients"

SCORE_TABLE_NAME = "Score"
EVALUATION_TABLE_NAME = "Validation Responses"
LATENCY_TABLE_NAME = "Model Latencies"
MODEL_NAME = "Finetuned-Review-Autocompletion"

In [4]:
with wandb.init(job_type="evaluation") as run:
    artifact = run.use_artifact('reviewco/Autocompletion with evaluation/finetuned-model:v1', type='model')
    artifact_dir = artifact.download()    
    base_llm = AutoModelForCausalLM.from_pretrained("facebook/opt-125m", device_map="auto", torch_dtype=torch.float16)
    tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m", local_files_only=True)
    model = PeftModel.from_pretrained(base_llm, artifact_dir,torch_dtype=torch.float16)
    model = model.merge_and_unload()
    
    # 評価を実行
    results = lm_eval.evaluator.simple_evaluate(
        model=model,
        tasks=["arc_easy","hellaswag","squad2"],  
        batch_size=16,
        num_fewshot=3,
        device="cuda"  
    )